# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(type(data))
data

<class 'dict'>


{'avgTotalVolume': 120034808,
 'calculationPrice': 'close',
 'change': 5.79,
 'changePercent': 0.04018,
 'close': 147.49,
 'closeSource': 'filifcoa',
 'closeTime': 1709248657066,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 149.85,
 'delayedPriceTime': 1725148650720,
 'extendedChange': -1.89,
 'extendedChangePercent': -0.01362,
 'extendedPrice': 148.08,
 'extendedPriceTime': 1683816177380,
 'high': 150.3,
 'highSource': 'at5mi pydcdirenu l ee1e',
 'highTime': 1735955512821,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 149.42,
 'iexCloseTime': 1672719395667,
 'iexLastUpdated': 1687680518071,
 'iexMarketPercent': 0.014885242184125753,
 'iexOpen': 143.19,
 'iexOpenTime': 1676636494890,
 'iexRealtimePrice': 146.03,
 'iexRealtimeSize': 105,
 'iexVolume': 1707488,
 'lastTradeTime': 1732320865753,
 'latestPrice': 147.94,
 'latestSource': 'Close',
 'latestTime': 'May 23, 2022',
 'latestUpdate': 1691010005781,
 'latestVolume': 11838

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
price

147.94

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
my_columns

['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for s in symbol.split(','):
        try:
            final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    s,
                    data[s]['quote']['latestPrice'],
                    data[s]['quote']['peRatio'],
                    'N/A'
                ], index = my_columns
            ), ignore_index = True)
        except KeyError:
            print(f'KeyError, {s} not found')

final_dataframe

/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

KeyError, DISCA not found


/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

...

/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

KeyError, VIAC not found
KeyError, WLTW not found


/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_123385/159688285.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

Ticker   Price Price-to-Earnings Ratio Number of Shares to Buy
0        A  129.94                   33.49                     N/A
1      AAL   17.38                    -4.6                     N/A
2      AAP     184                   19.94                     N/A
3     AAPL  147.46                    23.5                     N/A
4     ABBV  151.57                    22.1                     N/A
..     ...     ...                     ...                     ...
496    YUM  118.86                   20.66                     N/A
497    ZBH  123.58                  119.67                     N/A
498   ZBRA  336.92                   22.05                     N/A
499   ZION    56.2                    5.69                     N/A
500    ZTS     168                   38.92                     N/A

[501 rows x 4 columns]

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)

final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

Ticker   Price Price-to-Earnings Ratio Number of Shares to Buy
0    EBAY   45.98                    2.55                     N/A
1     NRG   46.89                     2.9                     N/A
2       F    13.4                    4.55                     N/A
3     HRB   33.61                     4.6                     N/A
4     NUE  130.32                    4.62                     N/A
5     COF   121.8                    4.77                     N/A
6     SYF    34.3                    4.79                     N/A
7     PVH   65.34                    5.02                     N/A
8     DHI   70.48                    5.05                     N/A
9     AIG   57.93                    5.12                     N/A
10    HPE   15.16                     5.4                     N/A
11    PHM   43.14                    5.42                     N/A
12   ZION    56.2                    5.69                     N/A
13    MPC   97.57                    5.77                     N/A
14    LEN   75.36                    5.88                     N/A
15   DISH   21.16                    6.01                     N/A
16    WBA   42.84                    6.07                     N/A
17    KSS   39.51                    6.18                     N/A
18     GM      36                    6.22                     N/A
19     GS  330.73                    6.32                     N/A
20    LYB  109.37                    6.36                     N/A
21      C   54.09                    6.42                     N/A
22    HPQ   36.06                    6.43                     N/A
23    DFS  113.43                    6.54                     N/A
24    WHR  176.68                    6.69                     N/A
25    APA    45.2                    6.78                     N/A
26    IVZ    19.5                    6.84                     N/A
27    BEN      26                    6.99                     N/A
28   INTC      43                    7.12                     N/A
29    BBY   73.42                    7.39                     N/A
30    DOW   67.95                    7.61                     N/A
31    KEY   18.96                    7.65                     N/A
32     WU   17.45                    7.74                     N/A
33    AIZ  186.77                    7.77                     N/A
34    LNC   54.36                     7.9                     N/A
35    UNM   36.49                     8.1                     N/A
36     CE  155.45                    8.13                     N/A
37    TSN   88.42                    8.13                     N/A
38     RE  286.21                     8.3                     N/A
39    PRU   104.3                    8.41                     N/A
40    MET   64.32                    8.48                     N/A
41    CFG   40.42                    8.54                     N/A
42     RF   21.47                     8.6                     N/A
43     MU   72.29                    8.82                     N/A
44    MOS      63                    8.96                     N/A
45    HBI   11.88                    9.03                     N/A
46      T   21.36                     9.1                     N/A
47    MHK  137.59                    9.19                     N/A
48   FANG  141.48                    9.35                     N/A
49    WFC      46                    9.42                     N/A

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:300000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe

Ticker   Price Price-to-Earnings Ratio Number of Shares to Buy
0    EBAY   45.98                    2.55                     130
1     NRG   46.89                     2.9                     127
2       F    13.4                    4.55                     447
3     HRB   33.61                     4.6                     178
4     NUE  130.32                    4.62                      46
5     COF   121.8                    4.77                      49
6     SYF    34.3                    4.79                     174
7     PVH   65.34                    5.02                      91
8     DHI   70.48                    5.05                      85
9     AIG   57.93                    5.12                     103
10    HPE   15.16                     5.4                     395
11    PHM   43.14                    5.42                     139
12   ZION    56.2                    5.69                     106
13    MPC   97.57                    5.77                      61
14    LEN   75.36                    5.88                      79
15   DISH   21.16                    6.01                     283
16    WBA   42.84                    6.07                     140
17    KSS   39.51                    6.18                     151
18     GM      36                    6.22                     166
19     GS  330.73                    6.32                      18
20    LYB  109.37                    6.36                      54
21      C   54.09                    6.42                     110
22    HPQ   36.06                    6.43                     166
23    DFS  113.43                    6.54                      52
24    WHR  176.68                    6.69                      33
25    APA    45.2                    6.78                     132
26    IVZ    19.5                    6.84                     307
27    BEN      26                    6.99                     230
28   INTC      43                    7.12                     139
29    BBY   73.42                    7.39                      81
30    DOW   67.95                    7.61                      88
31    KEY   18.96                    7.65                     316
32     WU   17.45                    7.74                     343
33    AIZ  186.77                    7.77                      32
34    LNC   54.36                     7.9                     110
35    UNM   36.49                     8.1                     164
36     CE  155.45                    8.13                      38
37    TSN   88.42                    8.13                      67
38     RE  286.21                     8.3                      20
39    PRU   104.3                    8.41                      57
40    MET   64.32                    8.48                      93
41    CFG   40.42                    8.54                     148
42     RF   21.47                     8.6                     279
43     MU   72.29                    8.82                      82
44    MOS      63                    8.96                      95
45    HBI   11.88                    9.03                     505
46      T   21.36                     9.1                     280
47    MHK  137.59                    9.19                      43
48   FANG  141.48                    9.35                      42
49    WFC      46                    9.42                     130

## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# Price-to-earnings ratio
peRatio = data[symbol]['quote']['peRatio']
# Price-to-book ratio
pbRatio = data[symbol]['advanced-stats']['priceToBook']
print(pbRatio)
# Price-to-sales ratio
psRatio = data[symbol]['advanced-stats']['priceToSales']
print(psRatio)
# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_ebitda = ev/ebitda
print(ev_ebitda)
# Enterprise Value divided by Gross Profit (EV/GP)

GP = data[symbol]['advanced-stats']['grossProfit']
ev_to_GP = ev/GP
print(ev_to_GP)



34.09
5.87
17.744403162183975
13.680990968237609


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]
rv_df = pd.DataFrame( columns = rv_columns )
for symbol in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for s in symbol.split(','):
        try:
            ev = data[s]['advanced-stats']['enterpriseValue']
            ebitda = data[s]['advanced-stats']['EBITDA']
            GP = data[s]['advanced-stats']['grossProfit']
            try:
                ev_ebitda = ev/ebitda
            #esto es para manejar algunos valores que no existian para EV o ebitda. Los vamos a dejar como NaN, 
            #y luego seran reemplazados por algo mas significativo
            except TypeError:
                ev_ebitda = np.NaN
            #pasa lo mismo con los valores de GP y EV
            try:
                ev_GP = ev/GP
            except TypeError:
                ev_GP = np.NaN
            
            rv_df = rv_df.append(
                pd.Series([
                    s,
                    data[s]['quote']['latestPrice'],
                    'N/A',
                    data[s]['quote']['peRatio'],
                    'N/A',
                    data[s]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[s]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_ebitda,
                    'N/A',
                    ev_GP,
                    'N/A',
                    'N/A'
                ], index =rv_columns
                ), ignore_index = True)
        except KeyError:
            print(f'Error, {s} not found')

rv_df

/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_12338

/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_12338

/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_12338

Error, DISCA not found


/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_12338

...

/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_12338

/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_12338

Error, VIAC not found
Error, WLTW not found


/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_123385/169364192.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
/tmp/ipykernel_12338

Ticker   Price Number of Shares to Buy Price-to-Earnings Ratio  \
0        A  126.58                     N/A                   32.34   
1      AAL   17.57                     N/A                   -4.79   
2      AAP   188.6                     N/A                      20   
3     AAPL  144.05                     N/A                   23.36   
4     ABBV  148.06                     N/A                   21.23   
..     ...     ...                     ...                     ...   
496    YUM  116.77                     N/A                      21   
497    ZBH  122.47                     N/A                  121.46   
498   ZBRA  346.33                     N/A                   22.36   
499   ZION    55.7                     N/A                     5.7   
500    ZTS     170                     N/A                   38.08   

    PE Percentile Price-to-Book Ratio PB Percentile Price-to-Sales Ratio  \
0             N/A                7.46           N/A                 6.05   
1             N/A               -1.21           N/A               0.3042   
2             N/A                3.64           N/A                 1.02   
3             N/A               33.27           N/A                 5.88   
4             N/A               17.21           N/A                  4.8   
..            ...                 ...           ...                  ...   
496           N/A               -3.94           N/A                  4.9   
497           N/A                2.06           N/A                 3.29   
498           N/A                6.01           N/A                 3.05   
499           N/A                1.36           N/A                  2.3   
500           N/A               16.54           N/A                  9.8   

    PS Percentile  EV/EBITDA EV/EBITDA Percentile      EV/GP EV/GP Percentile  \
0             N/A  23.382608                  N/A  11.126787              N/A   
1             N/A -23.552253                  N/A   1.040665              N/A   
2             N/A  10.749858                  N/A   2.359749              N/A   
3             N/A  18.091617                  N/A  14.314058              N/A   
4             N/A  11.853693                  N/A   8.346829              N/A   
..            ...        ...                  ...        ...              ...   
496           N/A  19.691584                  N/A   8.650686              N/A   
497           N/A  20.020618                  N/A   5.817561              N/A   
498           N/A  16.450047                  N/A   6.864848              N/A   
499           N/A   4.206932                  N/A   2.067115              N/A   
500           N/A  26.117054                  N/A  14.347717              N/A   

    RV Score  
0        N/A  
1        N/A  
2        N/A  
3        N/A  
4        N/A  
..       ...  
496      N/A  
497      N/A  
498      N/A  
499      N/A  
500      N/A  

[501 rows x 14 columns]

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
rv_df[rv_df.isnull().any(axis=1)]
len(rv_df[rv_df.isnull().any(axis=1)].index)
rv_df[rv_df.isnull().any(axis=1)]
#aqui se muestran las filas que llenamos anteriormente con N/A

Ticker    Price Number of Shares to Buy Price-to-Earnings Ratio  \
28    ALXN    184.9                     N/A                   61.86   
40     AON   277.34                     N/A                    None   
71   BRK.B    321.1                     N/A                    None   
118    CTL       11                     N/A                    9.97   
135  DISCK    24.46                     N/A                    None   
164   ETFC    50.02                     N/A                   14.48   
185   FLIR    58.18                     N/A                    33.2   
189    FOX    31.68                     N/A                    None   
192    FRT    110.8                     N/A                    None   
203   GOOG  2239.25                     N/A                    None   
323   MXIM   103.78                     N/A                   34.12   
324    MYL   15.941                     N/A                   31.17   
325    NBL     8.87                     N/A                   -0.75   
346    NWS    18.63                     N/A                    None   
358   PBCT    20.09                     N/A                   14.11   
440   TROW   123.64                     N/A                    None   
450     UA      8.7                     N/A                    None   
478   WELL     90.5                     N/A                    None   
491   XLNX   202.87                     N/A                   53.86   

    PE Percentile Price-to-Book Ratio PB Percentile Price-to-Sales Ratio  \
28            N/A                None           N/A                 None   
40            N/A                None           N/A                 None   
71            N/A                None           N/A                 None   
118           N/A                None           N/A                 None   
135           N/A                None           N/A                 None   
164           N/A                None           N/A                 None   
185           N/A                None           N/A                 None   
189           N/A                None           N/A                 None   
192           N/A                None           N/A                 None   
203           N/A                None           N/A                 None   
323           N/A                None           N/A                 None   
324           N/A                None           N/A                 None   
325           N/A                None           N/A                 None   
346           N/A                None           N/A                 None   
358           N/A                None           N/A                 None   
440           N/A                None           N/A                 None   
450           N/A                None           N/A                 None   
478           N/A                None           N/A                 None   
491           N/A                None           N/A                 None   

    PS Percentile EV/EBITDA EV/EBITDA Percentile EV/GP EV/GP Percentile  \
28            N/A       NaN                  N/A   NaN              N/A   
40            N/A       NaN                  N/A   NaN              N/A   
71            N/A       NaN                  N/A   NaN              N/A   
118           N/A       NaN                  N/A   NaN              N/A   
135           N/A       NaN                  N/A   NaN              N/A   
164           N/A       NaN                  N/A   NaN              N/A   
185           N/A       NaN                  N/A   NaN              N/A   
189           N/A       NaN                  N/A   NaN              N/A   
192           N/A       NaN                  N/A   NaN              N/A   
203           N/A       NaN                  N/A   NaN              N/A   
323           N/A       NaN                  N/A   NaN              N/A   
324           N/A       NaN                  N/A   NaN              N/A   
325           N/A       NaN                  N/A   NaN              N/A   
346         

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
#aqui, el tipo llena con la media de todos los valores q si existian, los valores que llenamos anteriormente con
#N/A, no se por que, me parece una pesima idea. Al menos, eso deberia hacer que si seleccionamos los 50 mejores,
#luego los que se llenaron con el promedio deberian quedar descartados
for column in ['Price-to-Earnings Ratio','Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA','EV/GP',]:
    rv_df[column].fillna(rv_df[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:
rv_df[rv_df.isnull().any(axis=1)]

Empty DataFrame
Columns: [Ticker, Price, Number of Shares to Buy, Price-to-Earnings Ratio, PE Percentile, Price-to-Book Ratio, PB Percentile, Price-to-Sales Ratio, PS Percentile, EV/EBITDA, EV/EBITDA Percentile, EV/GP, EV/GP Percentile, RV Score]
Index: []

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [16]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile',
}

for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[metric]] = stats.percentileofscore( rv_df[metric], rv_df.loc[row, metric])/100

rv_df        

Ticker   Price Number of Shares to Buy  Price-to-Earnings Ratio  \
0        A  126.58                     N/A                    32.34   
1      AAL   17.57                     N/A                    -4.79   
2      AAP   188.6                     N/A                    20.00   
3     AAPL  144.05                     N/A                    23.36   
4     ABBV  148.06                     N/A                    21.23   
..     ...     ...                     ...                      ...   
496    YUM  116.77                     N/A                    21.00   
497    ZBH  122.47                     N/A                   121.46   
498   ZBRA  346.33                     N/A                    22.36   
499   ZION    55.7                     N/A                     5.70   
500    ZTS     170                     N/A                    38.08   

    PE Percentile  Price-to-Book Ratio PB Percentile  Price-to-Sales Ratio  \
0        0.748503                 7.46      0.818363                6.0500   
1        0.033932                -1.21       0.05988                0.3042   
2        0.469062                 3.64      0.538922                1.0200   
3        0.558882                33.27      0.968064                5.8800   
4         0.51497                17.21       0.93014                4.8000   
..            ...                  ...           ...                   ...   
496      0.499002                -3.94      0.055888                4.9000   
497      0.986028                 2.06      0.292415                3.2900   
498      0.528942                 6.01      0.748503                3.0500   
499      0.071856                 1.36      0.156687                2.3000   
500      0.830339                16.54      0.926148                9.8000   

    PS Percentile  EV/EBITDA EV/EBITDA Percentile      EV/GP EV/GP Percentile  \
0        0.838323  23.382608             0.854291  11.126787         0.802395   
1        0.015968 -23.552253             0.007984   1.040665         0.035928   
2        0.147705  10.749858             0.339321   2.359749         0.135729   
3        0.828343  18.091617             0.708583  14.314058           0.9002   
4        0.751497  11.853693             0.397206   8.346829         0.644711   
..            ...        ...                  ...        ...              ...   
496      0.762475  19.691584             0.762475   8.650686         0.666667   
497      0.548902  20.020618             0.776447   5.817561         0.421158   
498       0.52495  16.450047             0.644711   6.864848         0.500998   
499      0.409182   4.206932             0.061876   2.067115         0.113772   
500      0.923154  26.117054             0.892216  14.347717         0.904192   

    RV Score  
0        N/A  
1        N/A  
2        N/A  
3        N/A  
4        N/A  
..       ...  
496      N/A  
497      N/A  
498      N/A  
499      N/A  
500      N/A  

[501 rows x 14 columns]

## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [17]:
from statistics import mean

for row in rv_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row, 'RV Score'] = mean(value_percentiles)

rv_df

Ticker   Price Number of Shares to Buy  Price-to-Earnings Ratio  \
0        A  126.58                     N/A                    32.34   
1      AAL   17.57                     N/A                    -4.79   
2      AAP   188.6                     N/A                    20.00   
3     AAPL  144.05                     N/A                    23.36   
4     ABBV  148.06                     N/A                    21.23   
..     ...     ...                     ...                      ...   
496    YUM  116.77                     N/A                    21.00   
497    ZBH  122.47                     N/A                   121.46   
498   ZBRA  346.33                     N/A                    22.36   
499   ZION    55.7                     N/A                     5.70   
500    ZTS     170                     N/A                    38.08   

    PE Percentile  Price-to-Book Ratio PB Percentile  Price-to-Sales Ratio  \
0        0.748503                 7.46      0.818363                6.0500   
1        0.033932                -1.21       0.05988                0.3042   
2        0.469062                 3.64      0.538922                1.0200   
3        0.558882                33.27      0.968064                5.8800   
4         0.51497                17.21       0.93014                4.8000   
..            ...                  ...           ...                   ...   
496      0.499002                -3.94      0.055888                4.9000   
497      0.986028                 2.06      0.292415                3.2900   
498      0.528942                 6.01      0.748503                3.0500   
499      0.071856                 1.36      0.156687                2.3000   
500      0.830339                16.54      0.926148                9.8000   

    PS Percentile  EV/EBITDA EV/EBITDA Percentile      EV/GP EV/GP Percentile  \
0        0.838323  23.382608             0.854291  11.126787         0.802395   
1        0.015968 -23.552253             0.007984   1.040665         0.035928   
2        0.147705  10.749858             0.339321   2.359749         0.135729   
3        0.828343  18.091617             0.708583  14.314058           0.9002   
4        0.751497  11.853693             0.397206   8.346829         0.644711   
..            ...        ...                  ...        ...              ...   
496      0.762475  19.691584             0.762475   8.650686         0.666667   
497      0.548902  20.020618             0.776447   5.817561         0.421158   
498       0.52495  16.450047             0.644711   6.864848         0.500998   
499      0.409182   4.206932             0.061876   2.067115         0.113772   
500      0.923154  26.117054             0.892216  14.347717         0.904192   

     RV Score  
0    0.812375  
1    0.030739  
2    0.326148  
3    0.792814  
4    0.647705  
..        ...  
496  0.549301  
497   0.60499  
498  0.589621  
499  0.162675  
500   0.89521  

[501 rows x 14 columns]

## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [18]:
rv_df.sort_values('RV Score', ascending = True, inplace = True)
rv_df = rv_df[:50]
rv_df.reset_index(drop = True, inplace = True)
rv_df

Ticker   Price Number of Shares to Buy  Price-to-Earnings Ratio  \
0     AAL   17.57                     N/A                    -4.79   
1     KSS    39.8                     N/A                     6.29   
2      GM      37                     N/A                     6.10   
3     AIG   58.36                     N/A                     5.15   
4     PVH   65.33                     N/A                     4.99   
5     UNM    36.9                     N/A                     7.94   
6     LNC   55.65                     N/A                     7.83   
7       F   12.85                     N/A                     4.59   
8     PRU   106.5                     N/A                     8.19   
9     SYF    34.1                     N/A                     4.67   
10    MET    65.7                     N/A                     8.52   
11      C   54.75                     N/A                     6.30   
12    WBA   42.66                     N/A                     6.10   
13    COF   120.4                     N/A                     4.69   
14    DXC   30.24                     N/A                   -12.95   
15    CAH   58.65                     N/A                   -17.00   
16      L    66.4                     N/A                    10.34   
17    ALL   125.8                     N/A                    10.72   
18    HPQ   35.27                     N/A                     6.54   
19    PHM   42.82                     N/A                     5.44   
20    APA    45.6                     N/A                     6.84   
21    HIG   69.29                     N/A                     9.50   
22    NRG      48                     N/A                     2.87   
23    AIZ  184.07                     N/A                     8.08   
24     RE  287.66                     N/A                     8.48   
25    DHI   68.99                     N/A                     4.89   
26    GPS   10.62                     N/A                    15.86   
27    LEN   77.25                     N/A                     5.94   
28    HCA  210.18                     N/A                     9.98   
29    BEN      26                     N/A                     7.00   
30    IVZ    18.8                     N/A                     6.85   
31    MHK  138.47                     N/A                     9.08   
32    NUE  126.48                     N/A                     4.58   
33     GS  328.77                     N/A                     6.39   
34      T   21.26                     N/A                     9.09   
35    UAA   10.11                     N/A                    12.91   
36    FDX  212.03                     N/A                    11.10   
37    WFC   44.94                     N/A                     9.59   
38    WHR  175.17                     N/A                     6.64   
39   DISH    21.3                     N/A                     6.14   
40    HPE   14.89                     N/A                     5.30   
41    TRV   173.2                     N/A                    11.10   
42   ZION    55.7                     N/A                     5.70   
43    UHS  122.64                     N/A                    10.75   
44    TSN   89.72                     N/A                     7.88   
45    AFL   58.32                     N/A                     9.36   
46   PRGO   38.63                     N/A                   -48.62   
47    WDC   59.17                     N/A                    10.09   
48    ALK   48.77                     N/A                    13.14   
49    KEY   19.09                     N/A                     7.64   

   PE Percentile  Price-to-Book Ratio PB Percentile  Price-to-Sales Ratio  \
0       0.033932              -1.2100       0.05988                0.3042   
1       0.083832               1.1300      0.122754                0.2682   
2       0.076846               0.8613      0.083832                0.4075   
3       0.063872               0.8144      0.075848                0.8410   
4       0.061876               0.8505       0.0

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

Enter the value of your portfolio:300000


In [20]:
position_size = float(portfolio_size)/len(rv_df.index)
for row in rv_df.index:
    rv_df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_df.loc[row, 'Price'])
rv_df

Ticker   Price Number of Shares to Buy  Price-to-Earnings Ratio  \
0     AAL   17.57                     341                    -4.79   
1     KSS    39.8                     150                     6.29   
2      GM      37                     162                     6.10   
3     AIG   58.36                     102                     5.15   
4     PVH   65.33                      91                     4.99   
5     UNM    36.9                     162                     7.94   
6     LNC   55.65                     107                     7.83   
7       F   12.85                     466                     4.59   
8     PRU   106.5                      56                     8.19   
9     SYF    34.1                     175                     4.67   
10    MET    65.7                      91                     8.52   
11      C   54.75                     109                     6.30   
12    WBA   42.66                     140                     6.10   
13    COF   120.4                      49                     4.69   
14    DXC   30.24                     198                   -12.95   
15    CAH   58.65                     102                   -17.00   
16      L    66.4                      90                    10.34   
17    ALL   125.8                      47                    10.72   
18    HPQ   35.27                     170                     6.54   
19    PHM   42.82                     140                     5.44   
20    APA    45.6                     131                     6.84   
21    HIG   69.29                      86                     9.50   
22    NRG      48                     125                     2.87   
23    AIZ  184.07                      32                     8.08   
24     RE  287.66                      20                     8.48   
25    DHI   68.99                      86                     4.89   
26    GPS   10.62                     564                    15.86   
27    LEN   77.25                      77                     5.94   
28    HCA  210.18                      28                     9.98   
29    BEN      26                     230                     7.00   
30    IVZ    18.8                     319                     6.85   
31    MHK  138.47                      43                     9.08   
32    NUE  126.48                      47                     4.58   
33     GS  328.77                      18                     6.39   
34      T   21.26                     282                     9.09   
35    UAA   10.11                     593                    12.91   
36    FDX  212.03                      28                    11.10   
37    WFC   44.94                     133                     9.59   
38    WHR  175.17                      34                     6.64   
39   DISH    21.3                     281                     6.14   
40    HPE   14.89                     402                     5.30   
41    TRV   173.2                      34                    11.10   
42   ZION    55.7                     107                     5.70   
43    UHS  122.64                      48                    10.75   
44    TSN   89.72                      66                     7.88   
45    AFL   58.32                     102                     9.36   
46   PRGO   38.63                     155                   -48.62   
47    WDC   59.17                     101                    10.09   
48    ALK   48.77                     123                    13.14   
49    KEY   19.09                     314                     7.64   

   PE Percentile  Price-to-Book Ratio PB Percentile  Price-to-Sales Ratio  \
0       0.033932              -1.2100       0.05988                0.3042   
1       0.083832               1.1300      0.122754                0.2682   
2       0.076846               0.8613      0.083832                0.4075   
3       0.063872               0.8144      0.075848                0.8410   
4       0.061876               0.8505       0.0

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [23]:
#writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
#rv_df.to_excel(writer, sheet_name = 'Value Strategy', index = False)

writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_df.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [26]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [27]:
column_formats = {
        'A': ['Ticker', string_template],
        'B': ['Price', dollar_template],
        'C': ['Number of Shares to Buy', integer_template],
        'D': ['Price-to-Earnings Ratio', float_template],
        'E': ['PE Percentile', percent_template],
        'F': ['Price-to-Book Ratio', float_template],
        'G': ['PB Percentile', percent_template],
        'H': ['Price-to-Sales Ratio', float_template],
        'I': ['PS Percentile', percent_template],
        'J': ['EV/EBITDA', float_template],
        'K': ['EV/EBITDA Percentile', percent_template],
        'L': ['EV/GP', float_template],
        'M': ['EV/GP Percentile', percent_template],
        'N': ['RV Score', string_template]
}

for c in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{c}:{c}', 25, column_formats[c][1])
    writer.sheets['Value Strategy'].write(f'{c}1', column_formats[c][0], column_formats[c][1])

writer.save()

## Saving Our Excel Output
As before, saving our Excel output is very easy: